# アーム数比較実験

このノートブックでは、LinBandit-BO（ECI_BO_Bandit_Randomに相当）において、生成するアーム（方向ベクトル）の数を変えて性能を比較します。

## 実験条件
- アーム数の設定：
  1. 1本
  2. 次元数の半分（10本、次元数20の場合）
  3. 次元数と同じ（20本、デフォルト）
  4. 次元数の2倍（40本）
  5. 次元数の3倍（60本）

- テスト関数: Styblinski-Tang, Rastrigin, Ackley
- coordinate_ratio = 0.8で固定

## 注意点
- 1本の場合は、coordinate_ratioに基づいて確率的に座標方向かランダム方向を選択
- アーム数が増えても、coordinate_ratioに基づいて座標方向とランダム方向の比率を維持

In [1]:
import math
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch

# BoTorch / GPyTorch
from botorch import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

# デフォルトのdtypeをfloat32に設定
torch.set_default_dtype(torch.float32)

# プロット設定
plt.rcParams["figure.dpi"] = 100

import warnings
warnings.filterwarnings("ignore")

C:\Users\nomuyu\PycharmProjects\LinBandit-BO\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## テスト関数群（100次元中先頭5次元が有効）

In [2]:
def styblinski_tang_100d(x, noise_std=1e-5):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    x5 = x[..., :5]
    res = 0.5 * torch.sum(x5**4 - 16.0*x5**2 + 5.0*x5, dim=-1)
    return res + torch.randn_like(res) * noise_std

def rastrigin_100d(x, noise_std=1e-5):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    x5 = x[..., :5]
    s = torch.sum(x5**2 - 10.0*torch.cos(2*math.pi*x5) + 10.0, dim=-1)
    return s + torch.randn_like(s) * noise_std

def ackley_100d(x, noise_std=1e-5):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    x5 = x[..., :5]
    d = 5
    sum_sq = torch.sum(x5**2, dim=-1)
    r = torch.sqrt(sum_sq / d)
    part1 = -20.0 * torch.exp(-0.2 * r)
    part2 = -torch.exp(torch.mean(torch.cos(2.0*math.pi*x5), dim=-1))
    res = part1 + part2 + 20.0 + math.e
    return res + torch.randn_like(res) * noise_std

## LinBandit-BOクラス（アーム数を可変にする拡張版）

In [3]:
class LinBanditBO_VariableArms:
    """LinBandit-BO with variable number of arms for experimental comparison"""
    
    def __init__(self, X, objective_function, bounds, n_initial, n_max, dim,
                 algo_base_name="LinBanditBO_VariableArms", coordinate_ratio=0.8, 
                 run_id=1, output_base_dir="output_results", n_arms=None):
        """
        Parameters
        ----------
        n_arms : int or None
            Number of arms to generate. If None, defaults to dim (standard behavior)
        """
        self.X = X.float()
        self.dim = dim
        self.n_arms = n_arms if n_arms is not None else dim  # 新しいパラメータ
        self.num_arms = dim
        self.A = torch.eye(dim)
        self.b = torch.zeros(dim)
        self.objective_function = objective_function
        self.bounds = bounds.float()
        self.n_initial = n_initial
        self.n_max = n_max
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []
        self.selected_direction_history = []
        self.theta_history = []
        self.coordinate_ratio = coordinate_ratio
        self.scale_init = 1.0
        self.run_id = run_id

        # アーム数をファイル名に含める
        arms_ratio = self.n_arms / self.dim
        self.function_name_with_ratio = f"Arms_{arms_ratio:.1f}x_coord_{self.coordinate_ratio:.1f}"
        self.algo_name_for_run = f"{algo_base_name}_{self.function_name_with_ratio}_run{self.run_id}"

        self.output_dir = os.path.join(output_base_dir, algo_base_name, self.function_name_with_ratio)
        os.makedirs(self.output_dir, exist_ok=True)

        self.total_iterations_for_bandit = 0

    def update_model(self):
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1], dtype=torch.float32),
            dtype=torch.float32, noise_constraint=1e-3
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)

    def initialize(self):
        y_val = self.objective_function(self.X)
        self.Y = y_val.unsqueeze(-1).float()
        y_max, y_min = self.Y.max().item(), self.Y.min().item()
        self.scale_init = (y_max - y_min) if (y_max - y_min) != 0 else 1.0
        self.update_model()
        post_mean = self.model.posterior(self.X).mean.squeeze(-1)
        bi = post_mean.argmin()
        self.best_value = post_mean[bi].item()
        self.best_point = self.X[bi]
        self.eval_history = [self.best_value] * self.n_initial

    def generate_arms(self):
        """修正版：アーム数を可変にしたgenerate_arms"""
        
        # 特殊ケース：アームが1本の場合
        if self.n_arms == 1:
            # coordinate_ratioに基づいて確率的に座標方向かランダム方向を選択
            if np.random.rand() < self.coordinate_ratio:
                # 座標方向を1つランダムに選択
                idx = np.random.choice(self.dim)
                e = torch.zeros(self.dim, device=self.X.device)
                e[idx] = 1.0
                return e.unsqueeze(0)
            else:
                # ランダム方向を生成
                rand_arm = torch.randn(self.dim, device=self.X.device)
                norm = rand_arm.norm()
                if norm > 1e-9:
                    rand_arm = rand_arm / norm
                return rand_arm.unsqueeze(0)
        
        # 通常のケース：複数アーム
        num_coord = int(self.coordinate_ratio * self.n_arms)
        num_coord = min(num_coord, self.dim)  # 座標方向は最大でも次元数まで
        
        # 座標方向の生成
        if num_coord > 0:
            # 座標をランダムに選択（LinBandit-BOの特徴）
            if num_coord <= self.dim:
                idxs = np.random.choice(self.dim, num_coord, replace=False)
            else:
                # num_coordがdimより大きい場合は、重複を許可
                idxs = np.random.choice(self.dim, num_coord, replace=True)
            
            coords = []
            for i in idxs:
                e = torch.zeros(self.dim, device=self.X.device)
                e[i] = 1.0
                coords.append(e)
                
            coord_arms = torch.stack(coords, 0)
        else:
            coord_arms = torch.zeros(0, self.dim, device=self.X.device)
        
        # ランダム方向の生成
        num_rand = self.n_arms - num_coord
        if num_rand > 0:
            rand_arms = torch.randn(num_rand, self.dim, device=self.X.device)
            norms = rand_arms.norm(dim=1, keepdim=True)
            rand_arms = torch.where(norms > 1e-9, rand_arms / norms, 
                                   torch.randn_like(rand_arms) / (torch.randn_like(rand_arms).norm(dim=1,keepdim=True)+1e-9))
        else:
            rand_arms = torch.zeros(0, self.dim, device=self.X.device)
            
        return torch.cat([coord_arms, rand_arms], 0)

    def select_arm(self, total_iterations_for_bandit):
        A_inv = torch.inverse(self.A)
        theta = A_inv @ self.b
        self.theta_history.append(theta.clone())

        current_round_t = total_iterations_for_bandit
        if current_round_t == 0: current_round_t = 1

        log_term_numerator = 1 + (current_round_t -1) * self.L**2 / self.lambda_reg
        if log_term_numerator <=0: log_term_numerator = 1e-9

        beta_t = (self.sigma * math.sqrt(
                    self.dim * math.log(log_term_numerator / self.delta))
                  + math.sqrt(self.lambda_reg)*self.S)

        pvals = []
        for i in range(self.arms_features.shape[0]):
            x = self.arms_features[i].view(-1,1)
            m = (theta.view(1,-1) @ x).item()
            try:
                v = (x.t() @ A_inv @ x).item()
            except torch.linalg.LinAlgError:
                v = (x.t() @ torch.linalg.pinv(self.A) @ x).item()

            pvals.append(m + beta_t*math.sqrt(max(v, 0)))
        return int(np.argmax(pvals))

    def propose_new_x(self, direction):
        ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
        
        active_dims_mask = direction.abs() > 1e-9
        if not active_dims_mask.any():
            lb, ub = -1.0, 1.0
        else:
            ratios_lower = (self.bounds[0] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))
            ratios_upper = (self.bounds[1] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))

            t_bounds = torch.zeros(self.dim, 2, device=self.X.device)
            t_bounds[:, 0] = torch.minimum(ratios_lower, ratios_upper)
            t_bounds[:, 1] = torch.maximum(ratios_lower, ratios_upper)

            lb = -float('inf')
            ub = float('inf')
            for i in range(self.dim):
                if active_dims_mask[i]:
                    lb = max(lb, t_bounds[i,0].item())
                    ub = min(ub, t_bounds[i,1].item())

        if lb > ub:
            lb, ub = -1.0, 1.0
            if self.best_point is not None:
                domain_width = (self.bounds[1,0] - self.bounds[0,0]).item()
                lb = -0.1 * domain_width
                ub =  0.1 * domain_width

        one_d_bounds = torch.tensor([[lb],[ub]], dtype=torch.float32, device=self.X.device)

        def ei_on_line(t_scalar_tensor):
            t_values = t_scalar_tensor.squeeze(-1)
            points_on_line = self.best_point.unsqueeze(0) + t_values.reshape(-1,1) * direction.unsqueeze(0)
            points_on_line_clamped = torch.clamp(points_on_line, self.bounds[0].unsqueeze(0), self.bounds[1].unsqueeze(0))
            return ei(points_on_line_clamped.unsqueeze(1))

        cand_t, acq_val_t = optimize_acqf(
            ei_on_line,
            bounds=one_d_bounds,
            q=1,
            num_restarts=10,
            raw_samples=100
        )

        alpha_star = cand_t.item()
        new_x = self.best_point + alpha_star * direction
        new_x_clamped = torch.clamp(new_x, self.bounds[0], self.bounds[1])

        return new_x_clamped, alpha_star, acq_val_t.item(), lb, ub

    def optimize(self):
        self.sigma = 1.0
        self.L = 1.0
        self.lambda_reg = 1.0
        self.delta = 0.1
        self.S = 1.0

        self.initialize()
        n_bo_iter = self.n_initial

        while n_bo_iter < self.n_max:
            self.total_iterations_for_bandit += 1
            self.arms_features = self.generate_arms()

            sel_idx = self.select_arm(self.total_iterations_for_bandit)
            direction = self.arms_features[sel_idx]
            self.selected_direction_history.append(direction.clone())

            new_x, _, _, _, _ = self.propose_new_x(direction)

            with torch.no_grad():
                predicted_mean_at_new_x = self.model.posterior(new_x.unsqueeze(0)).mean.squeeze().item()

            actual_y_at_new_x = self.objective_function(new_x.unsqueeze(0)).squeeze().item()

            prediction_error = abs(predicted_mean_at_new_x - actual_y_at_new_x)
            reward = 10.0 * (1.0 - math.exp(-prediction_error / self.scale_init))

            x_arm_for_update = direction.view(-1,1)
            self.A += x_arm_for_update @ x_arm_for_update.t()
            self.b += reward * direction

            self.X = torch.cat([self.X, new_x.unsqueeze(0)], 0)
            self.Y = torch.cat([self.Y, torch.tensor([[actual_y_at_new_x]], dtype=torch.float32, device=self.X.device)], 0)

            self.update_model()

            with torch.no_grad():
                posterior_mean_overall = self.model.posterior(self.X).mean.squeeze(-1)

            current_best_idx = posterior_mean_overall.argmin()
            self.best_value = posterior_mean_overall[current_best_idx].item()
            self.best_point = self.X[current_best_idx]

            self.eval_history.append(self.best_value)
            n_bo_iter += 1

        return self.best_point, self.best_value

## 実験実行

In [4]:
def generate_initial_points(n_initial, dim, bounds):
    return torch.rand(n_initial, dim) * (bounds[1] - bounds[0]) + bounds[0]

In [5]:
if __name__ == "__main__":
    test_funcs = [
        ("StyblinskiTang", styblinski_tang_100d, -195.83),
        ("Rastrigin", rastrigin_100d, 0.0),
        ("Ackley", ackley_100d, 0.0),
    ]
    dim = 20
    bounds = torch.tensor([[-5.0]*dim, [5.0]*dim], dtype=torch.float32)
    n_initial = 5
    n_iter = 300  
    n_runs = 20

    output_base_dir = "output_results_arm_comparison"
    os.makedirs(output_base_dir, exist_ok=True)

    coordinate_ratio = 0.8

    # アーム数の設定（1本、次元数の半分、次元数、次元数の2倍、次元数の3倍）
    arm_configs = [
        ("1_arm", 1),
        ("0.5x_arms", dim // 2),
        ("1x_arms", dim),
        ("2x_arms", dim * 2),
        ("3x_arms", dim * 3)
    ]

    # 全実行で共通の初期点
    initial_points_all_runs = [
        generate_initial_points(n_initial, dim, bounds)
        for _ in range(n_runs)
    ]

    for func_name_short, func_eval, global_opt_val in test_funcs:
        print(f"========== テスト関数実行中: {func_name_short} ==========")

        # 全アーム設定の結果を保存
        all_arm_results = {}

        for arm_name, n_arms in arm_configs:
            print(f"--- {arm_name} ({n_arms}本のアーム) 実行中 ---")
            
            histories_for_this_config = []
            dim_sums_for_this_config = []

            # tqdmの代わりにシンプルなプログレス表示を使用
            for run_idx in range(n_runs):
                print(f"\r  実行中: {run_idx + 1}/{n_runs}", end="", flush=True)
                
                initial_X_for_run = initial_points_all_runs[run_idx].clone().to(dtype=torch.float32)

                optimizer = LinBanditBO_VariableArms(
                    X=initial_X_for_run,
                    objective_function=func_eval,
                    bounds=bounds,
                    n_initial=n_initial,
                    n_max=n_iter,
                    dim=dim,
                    algo_base_name=func_name_short,
                    coordinate_ratio=coordinate_ratio,
                    run_id=run_idx + 1,
                    output_base_dir=output_base_dir,
                    n_arms=n_arms  # アーム数を指定
                )

                _, _ = optimizer.optimize()

                histories_for_this_config.append(optimizer.eval_history)

                if optimizer.selected_direction_history:
                    directions_tensor = torch.stack(optimizer.selected_direction_history, 0)
                    abs_sum_per_dim = directions_tensor.abs().sum(dim=0).cpu().numpy()
                    dim_sums_for_this_config.append(abs_sum_per_dim)
                else:
                    dim_sums_for_this_config.append(np.zeros(dim))
            
            print()  # 改行

            # 収束統計の計算
            eval_histories_np_array = np.array(histories_for_this_config)
            mean_convergence = eval_histories_np_array.mean(axis=0)
            std_convergence = eval_histories_np_array.std(axis=0)

            if dim_sums_for_this_config:
                avg_dim_abs_sum = np.mean(np.stack(dim_sums_for_this_config, 0), axis=0)
            else:
                avg_dim_abs_sum = np.zeros(dim)

            all_arm_results[arm_name] = {
                'mean_hist': mean_convergence,
                'std_hist': std_convergence,
                'avg_dim_abs_sum': avg_dim_abs_sum,
                'n_arms': n_arms
            }

        # 比較収束プロットの作成
        plt.figure(figsize=(14, 8))
        iters_plot = np.arange(1, n_iter + 1)
        
        colors = ['purple', 'blue', 'green', 'orange', 'red']
        for i, (arm_name, results) in enumerate(all_arm_results.items()):
            plt.plot(iters_plot, results['mean_hist'], 
                    label=f"{arm_name} ({results['n_arms']}本)", 
                    color=colors[i], linewidth=2)
            plt.fill_between(iters_plot,
                           results['mean_hist'] - results['std_hist'],
                           results['mean_hist'] + results['std_hist'],
                           alpha=0.15, color=colors[i])

        plt.axhline(global_opt_val, color='black', linestyle='--', label='大域最適値', linewidth=2)
        plt.xlabel("評価回数", fontsize=14)
        plt.ylabel("発見された最良目的値 (平均 ± 標準偏差)", fontsize=14)
        plt.title(f"{func_name_short}でのアーム数比較\n(coordinate_ratio={coordinate_ratio:.1f})", fontsize=16)
        plt.legend(fontsize=11, loc='upper right')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()

        # 比較プロットの保存
        comparison_plot_save_dir = os.path.join(output_base_dir, func_name_short)
        os.makedirs(comparison_plot_save_dir, exist_ok=True)
        plt.savefig(os.path.join(comparison_plot_save_dir, f"{func_name_short}_arm_comparison.png"), dpi=150)
        plt.close()

        # 方向比較プロットの作成（2行3列のサブプロット）
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for i, (arm_name, results) in enumerate(all_arm_results.items()):
            ax = axes[i]
            ax.bar(np.arange(dim), results['avg_dim_abs_sum'], alpha=0.7, color=colors[i])
            ax.set_xlabel("次元インデックス", fontsize=10)
            ax.set_ylabel("方向成分絶対値の平均和", fontsize=10)
            ax.set_title(f"{arm_name} ({results['n_arms']}本)\n{func_name_short}", fontsize=12)
            ax.set_xticks(np.arange(0, dim, step=max(1, dim//5)))
            ax.tick_params(axis='both', labelsize=8)
            ax.grid(axis='y', linestyle='--', alpha=0.5)
        
        # 6番目のサブプロットは空なので非表示に
        axes[5].set_visible(False)
        
        plt.suptitle(f"アーム数による方向使用比較 - {func_name_short}", fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(comparison_plot_save_dir, f"{func_name_short}_direction_comparison.png"), dpi=150)
        plt.close()

        # 最終性能の棒グラフ
        plt.figure(figsize=(10, 6))
        arm_names_list = list(all_arm_results.keys())
        final_means = [all_arm_results[name]['mean_hist'][-1] for name in arm_names_list]
        final_stds = [all_arm_results[name]['std_hist'][-1] for name in arm_names_list]
        x_pos = np.arange(len(arm_names_list))
        
        bars = plt.bar(x_pos, final_means, yerr=final_stds, capsize=10, 
                       color=colors[:len(arm_names_list)], alpha=0.7)
        
        plt.axhline(global_opt_val, color='black', linestyle='--', label='大域最適値', linewidth=2)
        plt.xlabel("アーム数設定", fontsize=14)
        plt.ylabel("最終的な最良値 (平均 ± 標準偏差)", fontsize=14)
        plt.title(f"{func_name_short} - アーム数による最終性能比較", fontsize=16)
        plt.xticks(x_pos, [f"{name}\n({all_arm_results[name]['n_arms']}本)" for name in arm_names_list], 
                   fontsize=11)
        plt.yticks(fontsize=12)
        plt.legend(fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(comparison_plot_save_dir, f"{func_name_short}_final_performance.png"), dpi=150)
        plt.close()

        print(f"========== テスト関数完了: {func_name_short} ==========")

    print("全ての実験が完了しました。")

========== テスト関数実行中: StyblinskiTang ==========
--- 1_arm (1本のアーム) 実行中 ---
  実行中: 20/20
--- 0.5x_arms (10本のアーム) 実行中 ---
  実行中: 20/20
--- 1x_arms (20本のアーム) 実行中 ---
  実行中: 20/20
--- 2x_arms (40本のアーム) 実行中 ---
  実行中: 20/20
--- 3x_arms (60本のアーム) 実行中 ---
  実行中: 20/20
========== テスト関数完了: StyblinskiTang ==========
========== テスト関数実行中: Rastrigin ==========
--- 1_arm (1本のアーム) 実行中 ---
  実行中: 20/20
--- 0.5x_arms (10本のアーム) 実行中 ---
  実行中: 20/20
--- 1x_arms (20本のアーム) 実行中 ---
  実行中: 20/20
--- 2x_arms (40本のアーム) 実行中 ---
  実行中: 20/20
--- 3x_arms (60本のアーム) 実行中 ---
  実行中: 20/20
========== テスト関数完了: Rastrigin ==========
========== テスト関数実行中: Ackley ==========
--- 1_arm (1本のアーム) 実行中 ---
  実行中: 20/20
--- 0.5x_arms (10本のアーム) 実行中 ---
  実行中: 20/20
--- 1x_arms (20本のアーム) 実行中 ---
  実行中: 20/20
--- 2x_arms (40本のアーム) 実行中 ---
  実行中: 20/20
--- 3x_arms (60本のアーム) 実行中 ---
  実行中: 20/20
========== テスト関数完了: Ackley ==========
全ての実験が完了しました。


## 結果の分析と考察

実験完了後、以下の観点から分析を行います：

### 1. アーム数と収束速度
- アーム数が少ない場合（1本）と多い場合（3倍）で収束速度にどのような違いがあるか
- 最適なアーム数は次元数に対してどの程度か

### 2. 探索と活用のバランス
- アーム数が多いほど探索の幅が広がるが、計算コストも増加
- 1本の場合は非常に局所的な探索になる可能性

### 3. 方向選択の多様性
- アーム数が増えることで、より多様な方向が選択されるか
- 有効次元（0-4）への集中度はアーム数によってどう変わるか

### 4. テスト関数による違い
- 関数の特性（滑らかさ、多峰性など）によってアーム数の効果が異なるか

実験を実行して、これらの観点から結果を確認してください。